## 执行数据抽取

- 本模块可以层层叠加，中间通过 SQL 连接，最后去做计算，实现计算优化。同时本模块也支持即时数据查询，选中 抽取数据
- 模块 m1 + m2 创建了一个 DataSource
- 模块 m3，选中 抽取数据
- 查看 m3 的输出，返回是 DataSource
- 使用场景
    - 一般情况下，不需要即时抽取数据，组合模块最后会做执行优化
    - 比如对于一个特定 DataSource，我们添加一个过滤，不需要添加一个 数据抽取 模块，而实现立即执行
- DAI 文档: https://bigquant.com/wiki/doc/dai-PLSbc1SbZX
- 数据字典: https://bigquant.com/data/home

In [1]:
from bigmodule import M

# <aistudiograph>

# @module(position="105,259", comment='', comment_collapsed=True)
m1 = M.input_features_dai.v15(
    mode='表达式',
    expr='m_lag(cn_stock_bar1d.volume, 1) AS f',
    expr_tables='',
    expr_drop_na=True,
    sql='',
    extract_data=False
)

# @module(position="105,330", comment='', comment_collapsed=True)
m2 = M.extract_data_dai.v13(
    sql=m1.data,
    start_date='2024-01-01',
    start_date_bound_to_trading_date=False,
    end_date='2024-03-31',
    end_date_bound_to_trading_date=False,
    before_start_days=90,
    debug=False
)

# @module(position="216,451", comment='', comment_collapsed=True)
m3 = M.input_features_dai.v15(
    input_1=m2.data,
    mode='表达式',
    expr="""input_1.f
input_1.f * input_1.f AS f2""",
    expr_tables='',
    expr_drop_na=True,
    sql="""-- 使用DAI SQL获取数据，构建因子等，如下是一个例子作为参考
-- DAI SQL 语法: https://bigquant.com/wiki/doc/dai-PLSbc1SbZX#h-sql%E5%85%A5%E9%97%A8%E6%95%99%E7%A8%8B
-- 使用数据输入1/2/3里的字段: e.g. input_1.close, input_1.* EXCLUDE(date, instrument)

SELECT

    -- 在这里输入因子表达式
    -- DAI SQL 算子/函数: https://bigquant.com/wiki/doc/dai-PLSbc1SbZX#h-%E5%87%BD%E6%95%B0
    -- 数据&字段: 数据文档 https://bigquant.com/data/home

    c_rank(volume) AS rank_volume,
    close / m_lag(close, 1) as return_0,

    -- 日期和股票代码
    date, instrument
FROM
    -- 预计算因子 cn_stock_bar1d https://bigquant.com/data/datasources/cn_stock_bar1d
    cn_stock_bar1d
WHERE
    -- WHERE 过滤，在窗口等计算算子之前执行
    -- 剔除ST股票
    st_status = 0
QUALIFY
    -- QUALIFY 过滤，在窗口等计算算子之后执行，比如 m_lag(close, 3) AS close_3，对于 close_3 的过滤需要放到这里
    -- 去掉有空值的行
    COLUMNS(*) IS NOT NULL
-- 按日期和股票代码排序，从小到大
ORDER BY date, instrument
""",
    extract_data=True
)
# </aistudiograph>

[2024-04-24 12:51:33] [info     ] input_features_dai.v15 开始运行..
[2024-04-24 12:51:35] [info     ] expr mode
[2024-04-24 12:51:35] [info     ] input_features_dai.v15 运行完成[1.99s].
[2024-04-24 12:51:35] [info     ] extract_data_dai.v13 开始运行..
[2024-04-24 12:51:36] [info     ] data extracted: (310188, 3)
[2024-04-24 12:51:37] [info     ] extract_data_dai.v13 运行完成[1.68s].
[2024-04-24 12:51:37] [info     ] input_features_dai.v15 开始运行..
[2024-04-24 12:51:37] [info     ] expr mode
[2024-04-24 12:51:37] [info     ] extract data ..
[2024-04-24 12:51:37] [info     ] extracted (310188, 4).
[2024-04-24 12:51:38] [info     ] input_features_dai.v15 运行完成[1.03s].


In [2]:
m3.data.read()

,f,f2,date,instrument
0,11980800,143539568640000,2024-01-02,000010.SZ
1,14474700,209516940090000,2024-01-03,000010.SZ
2,10585200,112046459040000,2024-01-04,000010.SZ
3,7448800,55484621440000,2024-01-05,000010.SZ
4,11018500,121407342250000,2024-01-08,000010.SZ
...,...,...,...,...
310183,5331089,28420509925921,2024-03-25,688233.SH
310184,4217555,17787770178025,2024-03-26,688233.SH
310185,3293779,10848980100841,2024-03-27,688233.SH
310186,2810383,7898252606689,2024-03-28,688233.SH
